In [1]:
from OfflineEvaluator import OfflineEvaluator
from Preprocessor import Preprocessor
import utils
from STOPWORDS import STOP_WORDS

import pickle
import logging
import operator
import sys
import os
from collections import Counter
import itertools
from itertools import combinations
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import spacy
import re
import networkx as nx

In [2]:
# Sample course SA
dbnum = ##
num_weeks = 6
START_DATE = #datetime(...)
TEMPORAL_START_WEEK = 3
instructors = #[...]

week_stride = 1
ratio_read = 0.0
k = 10

In [3]:
path_clean = "cleaned_data"
path_sample = "sample"
# preprocess
WEIGHT_POST_LOOKUP = {1: 'created',2:'liked',3:'linked',4:'replied',5:'revisited',6:'read',7:'anonymously read'}
WEIGHT_USER_LOOKUP = {1:'liked',2:'linked',3:'replied',4:'read',5:'anonymously read'}
CUTOFF_WEIGHT = 5
EXPLICIT_THRESHOLD = 5
IMPLICIT_THRESHOLD = 10

In [4]:
os.chdir('cleaned_data/db%d'%dbnum)
PRE_SIMS = {}
with open('precomputed_item_similarities_use3.pickle', 'rb') as handle:
    PRE_SIMS = pickle.load(handle)

PRE_EMBS = {}
with open('precomputed_sen_embs.pickle', 'rb') as handle:
    PRE_EMBS = pickle.load(handle)

PRE_KWS = {}
with open('precomputed_keyword_lookups.pickle', 'rb') as handle:
    PRE_KWS = pickle.load(handle)
    
with open('pfilter.pickle', 'rb') as handle:
    pfilter = pickle.load(handle)

with open('post_create_dates_all.pickle', 'rb') as handle:
    post_create_dates_all = pickle.load(handle)
    
g = nx.read_graphml("content_graph-10.graphml")
mapping = {}
for u,d in g.nodes(data=True):
    if d['node_type']=='post':
        mapping[u] = int(u)
content_graph_1 = nx.relabel_nodes(g, mapping)

g = nx.read_graphml("content_graph-0.2.graphml")
mapping = {}
for u,d in g.nodes(data=True):
    if d['node_type']=='post':
        mapping[u] = int(u)
content_graph_2 = nx.relabel_nodes(g, mapping)

g = nx.read_graphml("content_graph-5.graphml")
mapping = {}
for u,d in g.nodes(data=True):
    if d['node_type']=='post':
        mapping[u] = int(u)
content_graph_3 = nx.relabel_nodes(g, mapping)
os.chdir('../..')

## Tune KCB

In [ ]:
''' Iterative Evaluation'''
kcb_results = {}
kcb_best_feature = {1:(15,3)}

for cutoff_week in range(2, num_weeks):
    print('Current Week: ', cutoff_week)
    logging.info('........................... Offline Evaluator Starts..................................')
    evaluator = OfflineEvaluator(WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, CUTOFF_WEIGHT, 
                                 EXPLICIT_THRESHOLD, IMPLICIT_THRESHOLD, START_DATE, verbose=0)
    evaluator.set_ground_truth_criterion('explicit_implicit')
    evaluator.preprocess(path_sample, dbnum, cutoff_week, week_stride)
    evaluator.fit(PRE_SIMS, PRE_EMBS, PRE_KWS, content_graph_2, instructors)
    
    '''
    Calculate post creation dates
    '''
    lst = []
    post_create_dates_all = {}
    ct = 0
    for u, df_temp in evaluator.train_post_inter.items(): #NOTE only use trian post inter only
        #print(u)
        df_temp = df_temp.loc[df_temp.weight==1,['NoteID','Weeknum']]    
        lst.append(df_temp.set_index('NoteID').to_dict()['Weeknum'])
        ct+=len(df_temp)
    for dic in lst:
        for key,v in dic.items():
            post_create_dates_all[key] = v


    #kcb_settings = ['tfidf+lsi', 'tokens_phrases', 15, 'averaging','explicit_implicit','cosine']
    kcb_results_cur_wk = {}
    for feature_size in [10,15,20]:
        for top in [3,5]:
            print('feature size',feature_size)
            print('top ratio',top)
            kcb_settings = ['KCB', 'tokens_phrases', feature_size, 'averaging','explicit_implicit','cosine', top]
            recs = evaluator.run_cbf_eval(k, kcb_settings)
            kcb_results_cur_wk[(feature_size, top)] = np.mean([v[1] for v in recs.values()])
            if (feature_size, top) == kcb_best_feature.get(cutoff_week-1):
                print("Used last week's best hyperparameters")
                kcb_results[cutoff_week] = [recs] # store the results based on last week's cross validation
                
    # get the best hyp selection from the current week, will be used for the next week
    kcb_best_feature[cutoff_week] = sorted(kcb_results_cur_wk.items(), key=lambda kv: kv[1])[::-1][:1][0][0]

In [7]:
with open('results/db%d/%d-kcb-eval1_cv.pickle'%(dbnum,dbnum),'wb') as handle:
    pickle.dump(kcb_results, handle)
with open('results/db%d/%d-kcb-eval1_cv_best.pickle'%(dbnum, dbnum),'wb') as handle:
    pickle.dump(kcb_best_feature, handle)

## Tune MCF

In [ ]:
import warnings
warnings.filterwarnings('ignore')
wmf_results = {}
wmf_best_feature = {1:15}

for cutoff_week in range(2, num_weeks):
    logging.info('........................... Offline Evaluator Starts..................................')
    evaluator = OfflineEvaluator(WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, CUTOFF_WEIGHT, 
                                 EXPLICIT_THRESHOLD, IMPLICIT_THRESHOLD, START_DATE, verbose=0)
    evaluator.set_ground_truth_criterion('explicit_implicit')
    evaluator.preprocess(path_sample, dbnum, cutoff_week, week_stride)
    evaluator.fit(PRE_SIMS, PRE_EMBS, PRE_KWS, content_graph_2, instructors)
    
    '''
    Calculate post creation dates
    '''
    lst = []
    post_create_dates_all = {}
    ct = 0
    for u, df_temp in evaluator.train_post_inter.items(): #NOTE only use trian post inter only
        #print(u)
        df_temp = df_temp.loc[df_temp.weight==1,['NoteID','Weeknum']]    
        lst.append(df_temp.set_index('NoteID').to_dict()['Weeknum'])
        ct+=len(df_temp)
    for dic in lst:
        for key,v in dic.items():
            post_create_dates_all[key] = v
        
    wmf_results_cur_wk = {}
    for alpha in [15, 20, 25, 30]:
        wmf_settings = [alpha] # mode, #neighbors
        recs = evaluator.run_als_eval(k, wmf_settings)
        wmf_results_cur_wk[alpha] = np.mean([v[1] for v in recs.values()])
        if alpha == wmf_best_feature.get(cutoff_week-1):
            print("Used last week's best hyperparameters")
            wmf_results[cutoff_week] = [recs] # store the results based on last week's cross validation        
    # get the best hyp selection from the current week, will be used for the next week
    wmf_best_feature[cutoff_week] = sorted(wmf_results_cur_wk.items(), key=lambda kv: kv[1])[::-1][:1][0][0]

In [9]:
os.chdir('results/db%d'%dbnum)
with open('%d-wmf-eval1_cv.pickle'%(dbnum),'wb') as handle:
    pickle.dump(wmf_results, handle)
with open('%d-wmf-eval1_cv_best.pickle'%(dbnum),'wb') as handle:
    pickle.dump(wmf_best_feature, handle)
os.chdir('../..')

## Tune PPR

In [ ]:
pure_ppr_results = {}
pure_ppr_best_feature = {1:0.85}

for cutoff_week in range(2, num_weeks):
    logging.info('........................... Offline Evaluator Starts..................................')
    evaluator = OfflineEvaluator(WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, CUTOFF_WEIGHT, 
                                 EXPLICIT_THRESHOLD, IMPLICIT_THRESHOLD, START_DATE, verbose=0)
    evaluator.set_ground_truth_criterion('explicit_implicit')
    evaluator.preprocess(path_sample, dbnum, cutoff_week, week_stride)
    evaluator.fit(PRE_SIMS, PRE_EMBS, PRE_KWS, content_graph_2, instructors)
    
    '''
    Calculate post creation dates
    '''
    lst = []
    post_create_dates_all = {}
    ct = 0
    for u, df_temp in evaluator.train_post_inter.items(): #NOTE only use trian post inter only
        #print(u)
        df_temp = df_temp.loc[df_temp.weight==1,['NoteID','Weeknum']]    
        lst.append(df_temp.set_index('NoteID').to_dict()['Weeknum'])
        ct+=len(df_temp)
    for dic in lst:
        for key,v in dic.items():
            post_create_dates_all[key] = v
        
    pure_ppr_results_cur_wk = {}
    for damp in [0.75, 0.8, 0.85]:
        print('damp',damp)
        recs = evaluator.run_pure_ppr_eval(k, damp)
        pure_ppr_results_cur_wk[damp] = np.mean([v[1] for v in recs.values()])
        if damp == pure_ppr_best_feature.get(cutoff_week-1):
            print("Used last week's best hyperparameters")
            pure_ppr_results[cutoff_week] = [recs] # store the results based on last week's cross validation        
    # get the best hyp selection from the current week, will be used for the next week
    pure_ppr_best_feature[cutoff_week] = sorted(pure_ppr_results_cur_wk.items(), 
                                                key=lambda kv: kv[1])[::-1][:1][0][0]

In [11]:
os.chdir('results/db%d'%dbnum)
with open('%d-pure_ppr-eval1_cv.pickle'%(dbnum),'wb') as handle:
    pickle.dump(pure_ppr_results, handle)
with open('%d-pure_ppr-eval1_cv_best.pickle'%(dbnum),'wb') as handle:
    pickle.dump(pure_ppr_best_feature, handle)
os.chdir('../..')

## Tune CSCLRec, CoPPR

In [ ]:
csclrec_results = {}
coppr_results = {}
csclrec_best_feature = {1:(0.75,7,0.9)}
coppr_best_feature = {1:(0.75,7,0.9)}

for cutoff_week in range(2, num_weeks):
    logging.info('........................... Offline Evaluator Starts..................................')
    evaluator = OfflineEvaluator(WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, CUTOFF_WEIGHT, 
                                 EXPLICIT_THRESHOLD, IMPLICIT_THRESHOLD, START_DATE, verbose=0)
    evaluator.set_ground_truth_criterion('explicit_implicit')
    evaluator.preprocess(path_sample, dbnum, cutoff_week, week_stride)
    evaluator.fit(PRE_SIMS, PRE_EMBS, PRE_KWS, content_graph_2, instructors)
    
    '''
    Calculate post creation dates
    '''
    lst = []
    post_create_dates_all = {}
    ct = 0
    for u, df_temp in evaluator.train_post_inter.items(): #NOTE only use trian post inter only
        df_temp = df_temp.loc[df_temp.weight==1,['NoteID','Weeknum']]    
        lst.append(df_temp.set_index('NoteID').to_dict()['Weeknum'])
        ct+=len(df_temp)
    for dic in lst:
        for key,v in dic.items():
            post_create_dates_all[key] = v
    
    csclrec_results_cur_wk = {}
    for temporal in [0.7,0.75,0.8]:
        for nn in [3,5,7,10]:
            for damp in [0.8,0.85,0.9]:
                print('temporal decay',temporal)
                print('nn',nn)
                print('damping factor',damp)
                csclrec_settings = {'sna':True, 'cbf':True, 'hybrid':[0.1, 1.0, 0.01],
                                 'temporal_decay':temporal, '#neighbours': nn,'damp':damp}
                recs, ups = evaluator.run_csclrec_eval(k, csclrec_settings, pfilter, 
                                                   post_create_dates_all, 
                                                   TEMPORAL_START_WEEK, 
                                                   ratio_read)
                csclrec_results_cur_wk[(temporal, nn, damp)] = np.mean([v[1] for v in recs.values()])
                if (temporal, nn, damp) == csclrec_best_feature.get(cutoff_week-1):
                    print("Used last week's best hyperparameters")
                    csclrec_results[cutoff_week] = [recs, ups] # store the results based on last week's cross validation

    # get the best hyp selection from the current week, will be used for the next week
    csclrec_best_feature[cutoff_week] = sorted(csclrec_results_cur_wk.items(), key=lambda kv: kv[1])[::-1][:1][0][0]
    
    coppr_results_cur_wk = {}
    for temporal in [0.7,0.75,0.8]:
        for nn in [3,5,7,10]:
            for damp in [0.8,0.85,0.9]:
                print('temporal decay',temporal)
                print('nn',nn)
                print('damping factor',damp)
                coppr_settings = {'sna':True, 'cbf':True, 'hybrid':[0.1, 1.0, 1.0, 1.0],
                                 'temporal_decay':temporal, '#neighbours': nn,'damp':damp}
                recs, ups = evaluator.run_coppr_eval(k, coppr_settings, pfilter, 
                                                   post_create_dates_all, 
                                                   TEMPORAL_START_WEEK, 
                                                   ratio_read)
                coppr_results_cur_wk[(temporal, nn, damp)] = np.mean([v[1] for v in recs.values()])
                if (temporal, nn, damp) == coppr_best_feature.get(cutoff_week-1):
                    print("Used last week's best hyperparameters")
                    coppr_results[cutoff_week] = [recs, ups] # store the results based on last week's cross validation

    # get the best hyp selection from the current week, will be used for the next week
    coppr_best_feature[cutoff_week] = sorted(coppr_results_cur_wk.items(), key=lambda kv: kv[1])[::-1][:1][0][0]

In [13]:
with open('results/db%d/%d-csclrec-eval1_cv.pickle'%(dbnum,dbnum),'wb') as handle:
    pickle.dump(csclrec_results, handle)
with open('results/db%d/%d-csclrec-eval1_cv_best.pickle'%(dbnum,dbnum),'wb') as handle:
    pickle.dump(csclrec_best_feature, handle)
with open('results/db%d/%d-coppr-eval1_cv.pickle'%(dbnum,dbnum),'wb') as handle:
    pickle.dump(coppr_results, handle)
with open('results/db%d/%d-coppr-eval1_cv_best.pickle'%(dbnum, dbnum),'wb') as handle:
    pickle.dump(coppr_best_feature, handle)

## RND, PPL, and SCB

In [ ]:
''' Iterative Evaluation'''

scb_results = {}
random_results = {}
ppl_results = {}

week_stride = 1
ratio_read = 0.0
k = 10    

for cutoff_week in range(2, num_weeks):
    logging.info('........................... Offline Evaluator Starts..................................')
    evaluator = OfflineEvaluator(WEIGHT_POST_LOOKUP, WEIGHT_USER_LOOKUP, CUTOFF_WEIGHT, 
                                 EXPLICIT_THRESHOLD, IMPLICIT_THRESHOLD, START_DATE, verbose=0)
    evaluator.set_ground_truth_criterion('explicit_implicit')
    evaluator.preprocess(path_sample, dbnum, cutoff_week, week_stride)
    evaluator.fit(PRE_SIMS, PRE_EMBS, PRE_KWS, content_graph_2, instructors)
    
    '''
    Calculate post creation dates
    '''
    lst = []
    post_create_dates_all = {}
    ct = 0
    for u, df_temp in evaluator.train_post_inter.items(): #NOTE only use trian post inter only
        #print(u)
        df_temp = df_temp.loc[df_temp.weight==1,['NoteID','Weeknum']]    
        lst.append(df_temp.set_index('NoteID').to_dict()['Weeknum'])
        ct+=len(df_temp)
    for dic in lst:
        for key,v in dic.items():
            post_create_dates_all[key] = v


    recs =  evaluator.run_random_eval(k)
    random_results[cutoff_week] = [recs]
            
    recs =  evaluator.run_popularity_eval(k)
    ppl_results[cutoff_week] = [recs]
    
    scb_settings = ['sentence_emb_precomputed', PRE_EMBS, PRE_SIMS, 'averaging','explicit_implicit','cosine']
    recs = evaluator.run_cbf_eval(k, scb_settings)
    scb_results[cutoff_week] = [recs]
    

In [16]:
method_lookup = {0:'RND', 1:'PPL', 2:'SCB'}

In [17]:
os.chdir('results/db%d'%dbnum)
for i, results in enumerate([random_results, ppl_results,scb_results]):
    with open('%d-%s-results_eval1.pickle'%(dbnum,method_lookup[i]),'wb') as handle:
        pickle.dump(results, handle)
os.chdir('../..')